In [7]:
# !pip install fredapi
# !pip install quandl


In [ ]:
import pandas as pd
from datetime import datetime

from fredapi import Fred
import quandl

In [2]:
# Load data from pickle file
df = pd.read_pickle('./data/raw_data1.pkl')
meeting_dates = df['date'].unique()
meeting_dates = pd.to_datetime(meeting_dates) - pd.DateOffset(days=1)

In [9]:
fred = Fred(api_key='bacbd35093fa858b1039e16e98d3225d')
quandl.ApiConfig.api_key = 'nV2kQusAxBWcx3HojUXt'

In [10]:
# list of series ids for the required economic indicators
series_ids = {
    'CPI': 'CPIAUCSL',  # Consumer Price Index for All Urban Consumers: All Items in U.S. City Average
    'PPI': 'PPIACO',  # Producer Price Index by Commodity: All Commodities
    'GDP': 'GDP',  # Gross Domestic Product
    'Unemployment Rate': 'UNRATE',  # Unemployment Rate
    'Retail sales': 'RSAFS',  # Retail and Food Services Sales
    'Building Permits': 'PERMIT',  # New Private Housing Units Authorized by Building Permits
    'Existing home sales': 'EXHOSLUSM495S',  # Existing Home Sales
    'New home sales': 'HSN1F',  # New One Family Houses Sold: United States
    'PCE Inflation': 'PCEPILFE',  # Personal Consumption Expenditures: Chain-type Price Index\
    'Federal Funds Lower Range':  'DFEDTARL',
    'Federal Funds Upper Range': 'DFEDTARU',  
    'Federal Funds Effective Rate': 'DFF',
    'GDP Based recession Indicator': 'JHGDPBRINDX'
}

quandle_series_ids = {
    'Manufacturing PMI': 'ISM/MAN_PMI',
    'Non Manufcaturing PMI': 'ISM/NONMAN_NMI'
}


### Pull FRED data

In [15]:
# Create a dataframe to store the data
data = pd.DataFrame(index=meeting_dates)

# Fetch the data for each series and add it to the dataframe
for name, series_id in series_ids.items():
    # Get the data series
    series = fred.get_series(series_id)
    
    # Reindex the series to the meeting dates, forward filling any missing values
    series = series.reindex(meeting_dates, method='ffill')
    
    # Add the series to the data frame
    data[series_id] = series

### Pull Quandle data

In [16]:

for name, series_id in quandle_series_ids.items():
    # Get the data series
    series = quandl.get(series_id)
    
    # Reindex the series to the meeting dates, forward filling any missing values
    series = series.reindex(meeting_dates, method='ffill')
    
    # Add the series to the data frame
    data[series_id] = series

In [17]:
df = data.reset_index().melt(id_vars='index', var_name='ticker', value_name='value')
df.rename(columns={'index': 'date'}, inplace=True)
df.head()

,date,ticker,value
0,1994-02-03,CPIAUCSL,146.7
1,1994-03-21,CPIAUCSL,147.1
2,1994-05-16,CPIAUCSL,147.5
3,1994-07-05,CPIAUCSL,148.4
4,1994-08-15,CPIAUCSL,149.0


In [30]:
df.to_csv('./data/macro_data.csv')

In [31]:
df['ticker'].value_counts()

CPIAUCSL          241
PPIACO            241
GDP               241
UNRATE            241
RSAFS             241
PERMIT            241
EXHOSLUSM495S     241
HSN1F             241
PCEPILFE          241
DFEDTARL          241
DFEDTARU          241
DFF               241
JHGDPBRINDX       241
ISM/MAN_PMI       241
ISM/NONMAN_NMI    241
Name: ticker, dtype: int64

## Data dictionary

In [32]:
# Create the data dictionary for FRED series
data_dictionary = {}
for name, series_id in series_ids.items():
    series_info = fred.get_series_info(series_id)
    data_dictionary[series_id] = {
        'name' : name,
        'src_definition': series_info['title'],
        'frequency': series_info['frequency_short'],
        'units': series_info['units_short'],
        'source': 'FRED'
    }


# Update the data dictionary for Quandl series
for name, series_id in quandle_series_ids.items():
    data_dictionary[series_id] = {
        'name' : name,
        'src_definition': name,
        'frequency': 'M',  # Assuming monthly data
        'units': 'Index',  # Assuming Index units
        'source': 'Quandl'
    }


df_dict = pd.DataFrame(data_dictionary).T
df_dict = df_dict.reset_index()

# Rename the index column to 'ticker'
df_dict.rename(columns={'index': 'ticker'}, inplace=True)

df_dict.head(20)


,ticker,name,src_definition,frequency,units,source
0,CPIAUCSL,CPI,Consumer Price Index for All Urban Consumers: ...,M,Index 1982-1984=100,FRED
1,PPIACO,PPI,Producer Price Index by Commodity: All Commodi...,M,Index 1982=100,FRED
2,GDP,GDP,Gross Domestic Product,Q,Bil. of $,FRED
3,UNRATE,Unemployment Rate,Unemployment Rate,M,%,FRED
4,RSAFS,Retail sales,Advance Retail Sales: Retail Trade and Food Se...,M,Mil. of $,FRED
5,PERMIT,Building Permits,New Privately-Owned Housing Units Authorized i...,M,Thous. of Units,FRED
6,EXHOSLUSM495S,Existing home sales,Existing Home Sales,M,Number of Units,FRED
7,HSN1F,New home sales,New One Family Houses Sold: United States,M,Thous.,FRED
8,PCEPILFE,PCE Inflation,Personal Consumption Expenditures Excluding Fo...,M,Index 2012=100,FRED
9,DFEDTARL,Federal Funds Lower Range,Federal Funds Target Range - Lower Limit,D,%,FRED
